# The 8-puzzle

## Standard Formulation

In [1]:
# Including path to previous directory in built-in variable sys.path

import sys

sys.path.append('../')

In [2]:
# Importing the standard formulation

import numpy as np
from puzzle import formulations  # formulation returns the default
from collections import deque

In [3]:
# Class with a search problem

class Problem:
    def __init__(self, initial, objective, actions, result):
        self.initial = initial
        self.objective = objective
        self.actions = actions
        self.result = result  # defines the transition model

In [4]:
# Defining the manhattan heuristic

def heuristic_manhattan(grid):
    objective_grid = formulations.objective_grid(3)

    distance = 0
    for x in range(1, 9):
        x0, y0 = np.where(grid == x)
        x0, y0 = x0[0], y0[0]
        
        x1, y1 = np.where(objective_grid == x)
        x1, y1 = x1[0], y1[0]
        
        distance += abs(x1 - x0) + abs(y1 - y0)

    return distance

In [5]:
# Class with a tree node

class Node:
    def __init__(self, state, cost, parent, action):
        self.state = state
        self.cost = cost
        self.parent = parent
        self.action = action
        self.f = self.cost + heuristic_manhattan(self.state)

    def __str__(self):  # representing the node
        return '\n\n'.join(['  '.join(map(str, grid)) for grid in self.state[0:2]]) + \
               '  >>>  ' + str(self.cost) + '\n\n' + \
               '\n\n'.join(['  '.join(map(str, grid)) for grid in self.state[2:]])

    @classmethod
    def child(cls, problem, parent, action):
        state = problem.result(parent.state, action)
    
        return cls(state, parent.cost + 1, parent, action)  # returning the child

    @property
    def solution(self):
        node = self;
        solution = []

        while node:
            solution.append(node)
            node = node.parent

        solution.reverse()

        return solution

In [6]:
# Creating new instances of the problem

grid1 = np.array([[3, 2, 7],
                  [4, 8, 1],
                  [6, 5, 0]])

grid2 = np.array([[3, 2, 7],
                  [4, 8, 0],
                  [6, 5, 1]])

grid3 = np.array([[3, 2, 0],
                  [4, 8, 7],
                  [6, 5, 1]])

grid4 = np.array([[7, 2, 4],
                  [5, 0, 6],
                  [8, 3, 1]])

puzzle_12_steps = Problem(Node(grid1, 0, None, None), formulations.won,
                          formulations.available_moves, formulations.move_grid)

puzzle_13_steps = Problem(Node(grid2, 0, None, None), formulations.won,
                          formulations.available_moves, formulations.move_grid)

puzzle_14_steps = Problem(Node(grid3, 0, None, None), formulations.won,
                          formulations.available_moves, formulations.move_grid)

puzzle_26_steps = Problem(Node(grid4, 0, None, None), formulations.won,
                          formulations.available_moves, formulations.move_grid)

## PriorityQueue:
- Implements a PriorityQueue via a min-heap tree:
 - In this binary tree the smallest element is always at the root
 - Elements must be objects that are compared by the f property

In [7]:
class MinHeap:
    def __init__(self):
        self.contents = []
        self.capacity = 0
        self.size = 0
        
    def __getitem__(self, index):
        return self.contents[index]

    def remove_min(self):
        if (self.size < 1):
            return None

        min = self.contents[0]
        self.contents[0] = self.contents[self.size-1]
        self.size -= 1

        self.min_heapify(0)

        return min
    
    def remove(self, index):
        self.contents.pop(index)
        
        self.capacity -= 1
        self.size -= 1

    def add(self, node):
        index = self.size
        
        if (self.capacity == self.size):
            self.contents.append(node)
            self.capacity += 1

        self.insert_node(index, node)
        self.size += 1

    def parent(self, i):
        return int((i - 1) / 2)

    def child_left(self, i):
        return i*2 + 1

    def child_right(self, i):
        return i*2 + 2
    
    def index(self, state):
        position = None
        
        for pos, grid in enumerate([node.state for node in self.contents]):
            if np.all(grid == state):
                position = pos
                break
        
        return position

    def swap_nodes(self, i, j):
        temp = self.contents[i]
        self.contents[i] = self.contents[j]
        self.contents[j] = temp

    def min_heapify(self, i):
        l = self.child_left(i)
        r = self.child_right(i)

        minimum = i

        if l < self.size and self.contents[i].f > self.contents[l].f:
            minimum = l

        if r < self.size and self.contents[minimum].f > self.contents[r].f:
            minimum = r

        if minimum != i:
            self.swap_nodes(i, minimum)
            self.min_heapify(minimum)

    def insert_node(self, i, node):
        self.contents[i] = node
        
        while i > 0 and self.contents[self.parent(i)].f > self.contents[i].f:
            self.swap_nodes(i, self.parent(i))
            i = self.parent(i)

    def list_nodes(self):
        return self.contents[:self.size]


class PriorityQueue:
    def __init__(self):
        self.heap = MinHeap()
        
    def __getitem__(self, node):
        index = self.heap.index(node.state)
        
        return self.heap[index]

    def remove_min(self):
        return self.heap.remove_min()

    def add(self, node):
        self.heap.add(node)

    def index(self, node):
        return self.heap.index(node.state)
        
    def remove(self, node):
        self.heap.remove(self.index(node))

    @property
    def list_nodes(self):
        return self.heap.list_nodes()

## A*

### Graph-search:

In [8]:
# Constants for current search status

SEARCH_NOT_STARTED = 0
SEARCH_STARTED = 1
SEARCH_FAIL = 2
SEARCH_SUCCESS = 3

# Class that will implement A* graph search
class ASGS:  # A* graph search
    def __init__(self, problem):
        self.problem = problem
        self.frontier = PriorityQueue()
        self.explored = deque([])
        self.situation = SEARCH_NOT_STARTED
        self.solution = []

    def step_search(self):
        # Root node initial check
        if self.situation == SEARCH_NOT_STARTED:  # only if the search did not fail or was successful
            self.frontier.add(self.problem.initial)
            self.situation = SEARCH_STARTED  # indicates that the search has started

        # Checking if the search process failed
        if self.situation == SEARCH_FAIL:
            print("Search process failed!")
            return

        # Checking if the search was successful
        if self.situation == SEARCH_SUCCESS:
            print("Solution already found!")
            return

        # Performing the search step
        node = self.frontier.remove_min()
        
        if not node:  # empty border ends the search
            self.situation = SEARCH_FAIL
            return
        
        if self.problem.objective(node.state):
            self.solution = node.solution
            self.situation = SEARCH_SUCCESS
            return

        self.explored.append(node.state)

        for action in self.problem.actions(node.state):
            child = Node.child(self.problem, node, action)
            
            if not self.frontier.index(child) and not self.explored_node(child.state):                
                self.frontier.add(child)
            elif self.frontier.index(child) and child.f < self.frontier[child].f:
                self.frontier.remove(child)
                self.frontier.add(child)

    def search(self):
        # Loop that performs breadth-first search
        while self.situation != SEARCH_FAIL and self.situation != SEARCH_SUCCESS:
            self.step_search()

        if self.situation == SEARCH_FAIL:
            print("Search process failed!")
        else:
            print("Solution found!")
        
    def explored_node(self, state):
        for state_explored in self.explored:
            if np.all(state == state_explored):
                return True
            
        return False

    @property
    def show_solution(self):
        if self.situation == SEARCH_SUCCESS:
            return '\n'.join([node.__str__() for node in self.solution]) + \
                  f'\nCost: {self.solution[-1].cost}'
    
        print("Solution still not found!")  

    @property
    def show_frontier(self):
        return '#'*15 + '\n' + \
               '\n'.join([node.__str__() for node in self.frontier]) + \
               '\n' + '#'*15

### Tree-search:

In [9]:
# Constants for current search status

SEARCH_NOT_STARTED = 0
SEARCH_STARTED = 1
SEARCH_FAIL = 2
SEARCH_SUCCESS = 3

# Class that will implement A* tree search
class ASTS:  # A* tree search
    def __init__(self, problem):
        self.problem = problem
        self.frontier = PriorityQueue()
        self.situation = SEARCH_NOT_STARTED
        self.solution = []

    def step_search(self):
        # Root node initial check
        if self.situation == SEARCH_NOT_STARTED:  # only if the search did not fail or was successful
            self.frontier.add(self.problem.initial)
            self.situation = SEARCH_STARTED  # indicates that the search has started

        # Checking if the search process failed
        if self.situation == SEARCH_FAIL:
            print("Search process failed!")
            return

        # Checking if the search was successful
        if self.situation == SEARCH_SUCCESS:
            print("Solution already found!")
            return

        # Performing the search step
        node = self.frontier.remove_min()
        
        if not node:  # empty border ends the search
            self.situation = SEARCH_FAIL
            return
        
        if self.problem.objective(node.state):
            self.solution = node.solution
            self.situation = SEARCH_SUCCESS
            return

        for action in self.problem.actions(node.state):
            self.frontier.add(Node.child(self.problem, node, action))

    def search(self):
        # Loop that performs breadth-first search
        while self.situation != SEARCH_FAIL and self.situation != SEARCH_SUCCESS:
            self.step_search()

        if self.situation == SEARCH_FAIL:
            print("Search process failed!")
        else:
            print("Solution found!")
        
    @property
    def show_solution(self):
        if self.situation == SEARCH_SUCCESS:
            return '\n'.join([node.__str__() for node in self.solution]) + \
                  f'\nCost: {self.solution[-1].cost}'
    
        print("Solution still not found!")  

    @property
    def show_frontier(self):
        return '#'*15 + '\n' + \
               '\n'.join([node.__str__() for node in self.frontier]) + \
               '\n' + '#'*15
    
    @property
    def actions(self):
        for action in [step.action for step in self.solution[1:]]:
            yield action

### Testing the different searches:

In [10]:
a_star_graph = ASGS(puzzle_12_steps)
a_star_tree = ASTS(puzzle_12_steps)

%time a_star_graph.search()
%time a_star_tree.search()

Solution found!
CPU times: user 60.9 ms, sys: 8.15 ms, total: 69.1 ms
Wall time: 63.7 ms
Solution found!
CPU times: user 29.1 ms, sys: 172 µs, total: 29.3 ms
Wall time: 23.6 ms


In [11]:
a_star_graph = ASGS(puzzle_13_steps)
a_star_tree = ASTS(puzzle_13_steps)

%time a_star_graph.search()
%time a_star_tree.search()

Solution found!
CPU times: user 77.4 ms, sys: 11.4 ms, total: 88.8 ms
Wall time: 79.7 ms
Solution found!
CPU times: user 37.1 ms, sys: 560 µs, total: 37.7 ms
Wall time: 33.4 ms


In [12]:
a_star_graph = ASGS(puzzle_14_steps)
a_star_tree = ASTS(puzzle_14_steps)

%time a_star_graph.search()
%time a_star_tree.search()

Solution found!
CPU times: user 118 ms, sys: 136 µs, total: 118 ms
Wall time: 109 ms
Solution found!
CPU times: user 47 ms, sys: 4.55 ms, total: 51.6 ms
Wall time: 43.3 ms


In [13]:
a_star_graph = ASGS(puzzle_26_steps)
a_star_tree = ASTS(puzzle_26_steps)

%time a_star_graph.search()
%time a_star_tree.search()

Solution found!
CPU times: user 1min 27s, sys: 160 ms, total: 1min 27s
Wall time: 1min 27s
Solution found!
CPU times: user 42.2 s, sys: 348 ms, total: 42.6 s
Wall time: 42.6 s
